# have vs love vs want

## 1. importar las librerías + csv con los datos de la encuesta. 

In [352]:
# importar librerias

import pandas as pd
import plotly.express as px  
import plotly.graph_objects as go

import webbrowser
from threading import Timer
import numpy as np
import dash
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc #bootstrap


#crear un dataframe con toda la informacion de la encuesta
#anyadido: path relativo
df_csv = pd.read_csv ('survey/survey_results_public2021.csv', index_col = [0]) # El indice sera la columna con el ID de la respuesta
df_csv #mostrar df ()

,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,...,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
ResponseId,,,,,,,,,,,,,,,,,,,,,
1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,NaN,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.0
2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,NaN,...,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,NaN,...,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,NaN,...,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,10,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,NaN,Appropriate in length,Easy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83435,I am a developer by profession,Employed full-time,United States of America,Texas,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",6,5,...,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,I have a concentration and/or memory disorder ...,Appropriate in length,Easy,160500.0
83436,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Benin,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",4,2,...,18-24 years old,Man,No,Straight / Heterosexual,Black or of African descent,None of the above,None of the above,Appropriate in length,Easy,3960.0
83437,I am a developer by profession,Employed full-time,United States of America,New Jersey,NaN,"Secondary school (e.g. American high school, G...",11 - 17 years,School,10,4,...,25-34 years old,Man,No,NaN,White or of European descent,None of the above,None of the above,Appropriate in length,Neither easy nor difficult,90000.0


## 2. Preprocesar datos.

Tratar las columnas/conjunto de datos para comenzar a crear los gráficos. En este caso Age1stcode

Para lidiar con rangos de edades, algunos de los cuales tienen texto, se va a calcular una nueva columna con la media de todos ellos. 


In [353]:
# copiamos elementos que no sean NaN
from collections import Counter
from itertools import chain

def loveHateWant (df, columna1, columna2):
    #df2=df[['DatabaseHaveWorkedWith','DatabaseWantToWorkWith']].dropna().copy()
    df2=df[[columna1, columna2]].dropna().copy()
    
    df3=pd.DataFrame()


    df3['love'] = [set(x[0].split(';')) & set(x[1].split(';')) for x in df2.values]
    df3['hate'] = [set(x[0].split(';')) - set(x[1].split(';')) for x in df2.values]
    df3['want'] = [set(x[1].split(';')) - set(x[0].split(';')) for x in df2.values] 


    df_counts = (pd.DataFrame([Counter(chain.from_iterable(df3[column]))
                    for column in df3.columns],
                    index=['love', 'hate', 'want'])
                    .fillna(0)
                    .T
                    .sort_index()
)

    
    return df_counts



loveHateWant (df_csv, 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith')


,love,hate,want
Cassandra,799,757,3428
Couchbase,419,506,1251
DynamoDB,2840,1549,3199
Elasticsearch,5291,2822,6353
Firebase,6394,3419,4382
IBM DB2,354,865,324
MariaDB,6618,3935,1558
Microsoft SQL Server,10219,5966,1175
MongoDB,11742,5071,8389
MySQL,18121,11498,2186


## 3. Grafico. 

En este caso, un diagrama de barras.

In [354]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport', #permite ser responsive en movil
                            'content': 'width=device-width, initial-scale=1.0, maximum-scale=1.2, minimum-scale=0.5,'}]
                )
app.layout = html.Div([
    
    dbc.Row([
            dbc.Tabs([
                dbc.Tab(label='Databases', tab_id='db',labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"),
                dbc.Tab(label='Languages', tab_id='lang', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Platform', tab_id='plat', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='webframe', tab_id='web', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Miscelaneous', tab_id='misc', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Tools', tab_id='tools', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                dbc.Tab(label='Collab', tab_id='colab', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"),
                ],
                id="tabs",
                active_tab="db",
            ),
        dbc.CardBody(html.P(id="tab-content", className="card-text")),
    ]
)
])



@app.callback(
    Output("tab-content", "children"), 
    Input("tabs", "active_tab"))
    
def tab_content(active_tab):
    
    df = pd.DataFrame()
    
    if active_tab is not None:
        if active_tab =="db":
            df = loveHateWant(df_csv, 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith')
            fig = px.bar(df, 
                 x = [c for c in df.columns],
                 y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want: Databases',
                 orientation = 'h',
             )
            #nombre de ejes
            fig.update_yaxes(title_text="Databases")
            fig.update_xaxes(title_text="# Responses")
            
            #quitar color y grid del grafico
            fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
            fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
            return dcc.Graph(figure=fig)
        elif active_tab == "lang":
            df = loveHateWant (df_csv, 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith')
            fig = px.bar(df, 
                 x = [c for c in df.columns],
                 y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want: Languages',
                 orientation = 'h',
             )
            #nombre de ejes
            fig.update_yaxes(title_text="Languages")
            fig.update_xaxes(title_text="# Responses")
            
            #quitar color y grid del grafico
            fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
            fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
            return dcc.Graph(figure=fig)
        elif active_tab == "plat":
            df = loveHateWant (df_csv, 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith')
            fig = px.bar(df, 
                 x = [c for c in df.columns],
                 y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want: Platforms',
                 orientation = 'h',
             )
            #nombre de ejes
            fig.update_yaxes(title_text="Platforms")
            fig.update_xaxes(title_text="# Responses")
            
            #quitar color y grid del grafico
            fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
            fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
            return dcc.Graph(figure=fig)
        
        elif active_tab == "web":
            df = loveHateWant (df_csv, 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith')
            fig = px.bar(df, 
                 x = [c for c in df.columns],
                 y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want: Webframes',
                 orientation = 'h',
             )
            #nombre de ejes
            fig.update_yaxes(title_text="WebFrames")
            fig.update_xaxes(title_text="# Responses")
            
            #quitar color y grid del grafico
            fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
            fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
            return dcc.Graph(figure=fig)
        elif active_tab == "misc":
            df = loveHateWant (df_csv, 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith')
            fig = px.bar(df, 
                 x = [c for c in df.columns],
                 y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want: Miscelaneous Tech',
                 orientation = 'h',
             )
            #nombre de ejes
            fig.update_yaxes(title_text="Miscelaneous Tech")
            fig.update_xaxes(title_text="# Responses")
            
            #quitar color y grid del grafico
            fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
            fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
            return dcc.Graph(figure=fig)
        elif active_tab == "tools":
            df = loveHateWant (df_csv, 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith')
            fig = px.bar(df, 
                 x = [c for c in df.columns],
                 y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want: Tools',
                 orientation = 'h',
             )
            #nombre de ejes
            fig.update_yaxes(title_text="Languages")
            fig.update_xaxes(title_text="# Responses")
            
            #quitar color y grid del grafico
            fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
            fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
            return dcc.Graph(figure=fig)
        elif active_tab == "colab":
            df = loveHateWant (df_csv, 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith')
            fig = px.bar(df, 
                 x = [c for c in df.columns],
                 y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want: Collab tools',
                 orientation = 'h',
             )
            #nombre de ejes
            fig.update_yaxes(title_text="Languages")
            fig.update_xaxes(title_text="# Responses")
            
            #quitar color y grid del grafico
            fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
            fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
            return dcc.Graph(figure=fig)
        
        
    return html.P("This shouldn't ever be displayed...")
    
    

## 4. run server

In [355]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
